In [16]:
config=3
# import module we'll need to import our custom module
from shutil import copyfile

# copy our file into the working directory (make sure it has .py suffix)
copyfile(src = "../input/fp2021/data.py", dst = "../working/data.py")
copyfile(src = "../input/fp2021/utils.py", dst = "../working/utils.py")
copyfile(src = "../input/fp2021/training.py", dst = "../working/training.py")
copyfile(src = "../input/fp2021/layers.py", dst = "../working/layers.py")
#copyfile(src = "../input/notes1/weights2.npy", dst = "../working/weights2.npy")
# import all our functions
#from data import load_data

'../working/layers.py'

In [17]:
!pip install pickle5
import pickle5 as pickle
import csv
import calendar
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.ticker import  MultipleLocator, FormatStrFormatter
from scipy.interpolate import splrep
from IPython.core.display import display_html
from keras.models import load_model
from utils import np_haversine, density_map, get_clusters, plot_embeddings
from data import load_data
from training import start_new_session, process_features, create_model

# Display plots inline
%matplotlib inline

# Fix random seed for reproducibility
np.random.seed(42)

In [18]:
clusters_cache = '/kaggle/input/fp2021/clusters.npy'
clusters=np.load(clusters_cache, mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')

In [19]:
metadata_cache = '/kaggle/input/fp2021/metadata.pickle'
with open(metadata_cache, 'rb') as handle:
    metadata = pickle.load(handle)

In [20]:
file='/kaggle/input/fp2021/train0.npy'
train0=np.load(file, mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')
file='/kaggle/input/fp2021/train1.npy'
train1=np.load(file, mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')
file='/kaggle/input/fp2021/train2.npy'
train2=np.load(file, mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')
file='/kaggle/input/fp2021/train3.npy'
train3=np.load(file, mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')
file='/kaggle/input/fp2021/train4.npy'
train4=np.load(file, mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')
file='/kaggle/input/fp2021/train5.npy'
train5=np.load(file, mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')
file='/kaggle/input/fp2021/train6.npy'
train6=np.load(file, mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')
train=[train0,train1,train2,train3,train4,train5,train6]

In [21]:
file='/kaggle/input/fp2021/validation0.npy'
validation0=np.load(file, mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')
file='/kaggle/input/fp2021/validation1.npy'
validation1=np.load(file, mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')
file='/kaggle/input/fp2021/validation2.npy'
validation2=np.load(file, mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')
file='/kaggle/input/fp2021/validation3.npy'
validation3=np.load(file, mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')
file='/kaggle/input/fp2021/validation4.npy'
validation4=np.load(file, mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')
file='/kaggle/input/fp2021/validation5.npy'
validation5=np.load(file, mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')
file='/kaggle/input/fp2021/validation6.npy'
validation6=np.load(file, mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')
validation=[validation0,validation1,validation2,validation3,validation4,validation5,validation6]

In [22]:
metadata_cache = '/kaggle/input/fp2021/metadata.pickle'
with open(metadata_cache, 'rb') as handle:
    metadata = pickle.load(handle)
tl_cache = '/kaggle/input/fp2021/train-labels.npy'
vl_cache = '/kaggle/input/fp2021/validation-labels.npy'
train_labels=np.load(tl_cache, mmap_mode=None, allow_pickle=True, fix_imports=True, encoding='ASCII')
validation_labels=np.load(vl_cache, mmap_mode=None, allow_pickle=True, fix_imports=True, encoding='ASCII')

In [23]:
#import pickle
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import scale
from keras.models import Sequential
from tensorflow.keras.optimizers import SGD, Adam, Adagrad
from keras.layers.embeddings import Embedding
from keras.layers.core import Dense, Reshape, Activation, Dropout
from keras.layers import *
from keras import backend as K
from keras.callbacks import ModelCheckpoint
from utils import tf_haversine
from data import load_data
from utils import get_clusters
from keras.models import Model

from tensorflow.compat.v1.keras.layers import LSTM   # CuDNNLSTM
import sys
sys.path.append('..')  # add parent directory to Python path for layers.py access
from layers import Attention, SelfAttention

def create_model(metadata,clusters,config):
    """
    Creates all the layers for our neural network model.
    """
      
    # Arbitrary dimension for all embeddings
    embedding_dim = 10

    # Quarter hour of the day embedding
    embed_quarter_hour = Sequential()
    embed_quarter_hour.add(Embedding(metadata['n_quarter_hours'], embedding_dim, input_length=1))
    embed_quarter_hour.add(Reshape((embedding_dim,1)))

    # Day of the week embedding
    embed_day_of_week = Sequential()
    embed_day_of_week.add(Embedding(metadata['n_days_per_week'], embedding_dim, input_length=1))
    embed_day_of_week.add(Reshape((embedding_dim,1)))

    # Week of the year embedding
    embed_week_of_year = Sequential()
    embed_week_of_year.add(Embedding(metadata['n_weeks_per_year'], embedding_dim, input_length=1))
    embed_week_of_year.add(Reshape((embedding_dim,1)))

    # Client ID embedding
    embed_client_ids = Sequential()
    embed_client_ids.add(Embedding(metadata['n_client_ids'], embedding_dim, input_length=1))
    embed_client_ids.add(Reshape((embedding_dim,1)))

    # Taxi ID embedding
    embed_taxi_ids = Sequential()
    embed_taxi_ids.add(Embedding(metadata['n_taxi_ids'], embedding_dim, input_length=1))
    embed_taxi_ids.add(Reshape((embedding_dim,1)))

    # Taxi stand ID embedding
    embed_stand_ids = Sequential()
    embed_stand_ids.add(Embedding(metadata['n_stand_ids'], embedding_dim, input_length=1))
    embed_stand_ids.add(Reshape((embedding_dim,1)))
    
    # GPS coordinates (5 first lat/long and 5 latest lat/long, therefore 20 values)
    coords = Sequential()
    coords.add(Dense(1, input_dim=20))

    # Merge all the inputs into a single input layer
    mergedOut = Add()([embed_quarter_hour.output,
                embed_day_of_week.output,
                embed_week_of_year.output,
                embed_client_ids.output,
                embed_taxi_ids.output,
                embed_stand_ids.output,
                coords.output])
    
    #encoder_output, hidden_state, cell_state = LSTM(50, activation='tanh',input_shape=(None,None, 1),
    #                                                return_sequences=True,return_state=True)(mergedOut)
    #attention_input = [encoder_output, hidden_state]
    lstm_layer = tf.keras.layers.LSTM(200, return_sequences=True,return_state=True)
    if config != 0:
        encoder_output, hidden_state, cell_state,  *_ = Bidirectional(lstm_layer,merge_mode="sum")(mergedOut)
        attention_input = [encoder_output, hidden_state]
    else:
        encoder_output = LSTM(units=64)(mergedOut)

    # Optional Attention Mechanisms
    if config == 1:
        encoder_output, attention_weights = SelfAttention(size=64,
                                                      num_hops=10,
                                                      use_penalization=False)(encoder_output)
    elif config == 2:
        encoder_output, attention_weights = Attention(context='many-to-one',
                                                  alignment_type='global')(attention_input)
        encoder_output = Flatten()(encoder_output)
    elif config == 3:
        encoder_output, attention_weights = Attention(context='many-to-one',
                                                  alignment_type='local-m',
                                                  window_width=10,
                                                  score_function='general')(attention_input)
        encoder_output = Flatten()(encoder_output)
        
    
    #encoder_output=Dense(1)(encoder_output)
    # Determine cluster probabilities using softmax
    mergedOut=Dense(len(clusters))(encoder_output)
    mergedOut=Activation('softmax')(mergedOut)

    # Final activation layer: calculate the destination as the weighted mean of cluster coordinates
    cast_clusters = K.cast_to_floatx(clusters)
    def destination(probabilities):
        return tf.matmul(probabilities, cast_clusters)
    mergedOut=Activation(destination)(mergedOut)

    newModel = Model([embed_quarter_hour.input,
                embed_day_of_week.input,
                embed_week_of_year.input,
                embed_client_ids.input,
                embed_taxi_ids.input,
                embed_stand_ids.input,
                coords.input], mergedOut)
    #use lists if you want more than one input or output  
    
    # Compile the model
    optimizer = SGD(lr=0.1, momentum=0.9, clipvalue=1.)  # Use `clipvalue` to prevent exploding gradients

    newModel.compile(loss=tf_haversine, optimizer=optimizer)
    
    return newModel

In [24]:
model=create_model(metadata,clusters,config)


In [25]:
class CustCallback(tf.keras.callbacks.Callback):
    def on_train_end(self, logs=None):
        global training_finished
        training_finished = True
    def on_epoch_end(self, epoch, logs=None):
        z=len(f)
        file_path="../working/Newweights"+str(z)
        weight=model.get_weights()
        np.save(file_path, weight)
        f.append(0)

In [26]:
class MyCallback(tf.keras.callbacks.Callback):
    def on_train_end(self, logs=None):
        global training_finished
        training_finished = True
    def on_epoch_end(self, epoch, logs=None):
        z=len(e)
        if z>len(g):
            a=model.history.history['val_loss'][z-len(g)-1]
            b=model.history.history['loss'][z-len(g)-1]
            c=float(a)-float(b)
            if c<0:
                print("Sebelumnya Underfitting")
            else:
                print("Sebelumnya Overfitting")
        e.append(0)
            

In [27]:
e=[]
f=[]
g=[]
h=[]
for i in range(80):
    f.append(0)
    e.append(0)
    g.append(0)

In [28]:
#file='../input/latb82fiturlocm/Newweights'+str(len(g)-1)+'.npy'
file='./Newweights'+str(len(g)-1)+'.npy'
y=np.load(file,allow_pickle=True)
model.set_weights(y)

In [29]:
# Estimate the GPS clusters
print("Estimating clusters...")
#clusters = get_clusters(data.train_labels)
n_epochs=100
batch_size=200
save_prefix='mymodel'


Estimating clusters...


In [ ]:
from keras.callbacks import ModelCheckpoint, CSVLogger
from keras.callbacks import EarlyStopping
 
callbacks = []
if save_prefix is not None:
        # Save the model's intermediary weights to disk after each epoch
    file_path="cache/%s-{epoch:03d}-{val_loss:.4f}.hdf5" % save_prefix
    checkpoint = ModelCheckpoint(file_path,monitor='val_loss',mode='auto',save_weights_only=True,verbose=0)
    #checkpoint = weight_save(model.get_weights(),b)
    callbacks.append(checkpoint)
    #callbacks.append(ModelCheckpoint(file_path, monitor='val_loss', mode='min', save_weights_only=True, verbose=1))
    #callbacks.append(ModelCheckpoint(file_path, monitor='val_loss', verbose=0, save_best_only=False, mode='auto'))
    #model.fit(X_train,y_train,batch_size=batch_size,nb_epoch=nb_epoch,callbacks=[weight_save_callback])
#g.append(0)
#epoch>1
#first epoch
#saver = weight_save(model.get_weights(),b)   
#es = EarlyStopping(monitor='val_loss', patience=0, verbose=1)
arc='AttenLSTMfitur91-b200-'+str(len(g))+'.log'
csv_logger = CSVLogger(arc, separator=',', append=False)
print("Creating model...")
start_new_session()
print("Train model...")
history=model.fit(train, train_labels,
        initial_epoch=len(g),epochs=100, batch_size=batch_size,
        validation_data=(validation, validation_labels)
        ,callbacks=[csv_logger,CustCallback(),MyCallback()])
t=0
while(t<1):
    t=t

Creating model...
Train model...


2022-01-20 14:25:23.114215: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-20 14:25:23.115589: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-20 14:25:23.116513: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-20 14:25:23.117693: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-20 14:25:23.118840: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

Epoch 81/100


2022-01-20 14:25:27.362480: W tensorflow/core/grappler/costs/op_level_cost_estimator.cc:689] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "GPU" vendor: "NVIDIA" model: "Tesla P100-PCIE-16GB" frequency: 1328 num_cores: 56 environment { key: "architecture" value: "6.0" } environment { key: "cuda" value: "11000" } environment { key: "cudnn" value: "8005" } num_registers: 65536 l1_cache_size: 24576 l2_cache_size: 4194304 shared_memory_size_per_multiprocessor: 65536 memory_size: 16152002560 bandwidth: 732160000 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }


6577/6578 [============================>.] - ETA: 0s - loss: 1.9024

2022-01-20 14:27:06.659498: W tensorflow/core/grappler/costs/op_level_cost_estimator.cc:689] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "GPU" vendor: "NVIDIA" model: "Tesla P100-PCIE-16GB" frequency: 1328 num_cores: 56 environment { key: "architecture" value: "6.0" } environment { key: "cuda" value: "11000" } environment { key: "cudnn" value: "8005" } num_registers: 65536 l1_cache_size: 24576 l2_cache_size: 4194304 shared_memory_size_per_multiprocessor: 65536 memory_size: 16152002560 bandwidth: 732160000 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }


6578/6578 [==============================] - 113s 17ms/step - loss: 1.9024 - val_loss: 2.0095
Epoch 82/100
6578/6578 [==============================] - 109s 17ms/step - loss: 1.9005 - val_loss: 2.0061
Sebelumnya Overfitting
Epoch 83/100
6578/6578 [==============================] - 107s 16ms/step - loss: 1.8997 - val_loss: 2.0176
Sebelumnya Overfitting
Epoch 84/100
6578/6578 [==============================] - 107s 16ms/step - loss: 1.8982 - val_loss: 2.0099
Sebelumnya Overfitting
Epoch 85/100
6578/6578 [==============================] - 107s 16ms/step - loss: 1.8978 - val_loss: 2.0199
Sebelumnya Overfitting
Epoch 86/100
6578/6578 [==============================] - 108s 16ms/step - loss: 1.8962 - val_loss: 2.0058
Sebelumnya Overfitting
Epoch 87/100
6578/6578 [==============================] - 107s 16ms/step - loss: 1.8959 - val_loss: 2.0078
Sebelumnya Overfitting
Epoch 88/100
6578/6578 [==============================] - 108s 16ms/step - loss: 1.8958 - val_loss: 2.0048
Sebelumnya Overfitt

In [ ]:
#metadata_cache = '/kaggle/input/dataset91/train.pickle'
#with open(metadata_cache, 'rb') as handle:
#    train = pickle.load(handle)
#metadata_cache = '/kaggle/input/dataset91/validation.pickle'
#with open(metadata_cache, 'rb') as handle:
#    validation = pickle.load(handle)
import pickle5 as pickle
metadata_cache = '/kaggle/input/fp2021/competition-test.pickle'
with open(metadata_cache, 'rb') as handle:
    competition_test = pickle.load(handle)
train_labels=np.load('/kaggle/input/fp2021/train-labels.npy')
validation_labels=np.load('/kaggle/input/fp2021/validation-labels.npy')
competition_test_labels=np.load('/kaggle/input/fp2021/competition-test-labels.npy')
print("HASIL EVALUASI MODEL-INPUT 82 Full GPS")
print(" ")
test_predictions = model.predict(process_features(competition_test))
print("Mean Haversine Test:{}".format(np_haversine(test_predictions, competition_test_labels).mean()))
validation_predictions = model.predict(validation)
print("Mean Haversine Validation:{}".format(np_haversine(validation_predictions, validation_labels).mean()))
train_predictions = model.predict(train)
print("Mean Haversine Train:{}".format(np_haversine(train_predictions, train_labels).mean()))
print(" ")
y_train=train_labels
y_train_pred=train_predictions
y_valid=validation_labels
y_valid_pred=validation_predictions
y_test=competition_test_labels
y_test_pred=test_predictions
from sklearn import metrics
print("MSE train:{}".format(metrics.mean_squared_error(y_train, y_train_pred)))
print("MSE validation:{}".format(metrics.mean_squared_error(y_valid, y_valid_pred)))
print("MSE test:{}".format(metrics.mean_squared_error(y_test, y_test_pred)))
print(" ")
print("MAE score train:{}".format(metrics.mean_absolute_error(y_train, y_train_pred)))
print("MAE score validation:{}".format(metrics.mean_absolute_error(y_valid, y_valid_pred)))
print("MAE score test:{}".format(metrics.mean_absolute_error(y_test, y_test_pred)))
print(" ")
print("R2 score train:{}".format(metrics.r2_score(y_train, y_train_pred)))
print("R2 score validation:{}".format(metrics.r2_score(y_valid, y_valid_pred)))
print("R2 score test:{}".format(metrics.r2_score(y_test, y_test_pred)))
